#  R(データミックスのインテグレーションステップ課題　エンジンの予知保全　クラスターごとの生存時間分析による予測.R)において生存時間分析で予測を行ったデータを読み込み、混合行列を作成する

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score, f1_score,classification_report
from random import randint, shuffle
import os
import seaborn as sns





In [2]:
test_data = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/test_data/test_data.csv')

# testデータのRUL（答え）をクラスターごとに分けて、他必要な処理を行ったファイルを読み込む

In [3]:
# clusterごとのtestcsvを読み込み
# testデータのRUL（答え）をクラスターごとに分けて、他必要な処理を行ったファイルを読み込む
test_data_RUL_cluster0 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_RUL_cluster0.csv')
test_data_RUL_cluster1 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_RUL_cluster1.csv')

# 閾値0.5

# Rで作成した予測付きのtestデータを読み込む

In [4]:
test_data_R_cluster0_0_5 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster0_0.5.csv')
test_data_R_cluster1_0_5 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster1_0.5.csv')

In [5]:
test_data_R_cluster0_0_5.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,0


In [6]:
test_data_R_cluster1_0_5.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,9,1,-0.0009,-0.0001,100,518.67,642.06,1590.15,1408.02,...,0.03,393,2388,100,38.90,23.3147,2,-0.119871,0.470068,0
1,2,9,2,0.0031,-0.0002,100,518.67,642.74,1584.08,1413.33,...,0.03,392,2388,100,38.70,23.3071,3,0.035570,0.508891,1
2,3,9,3,0.0023,-0.0001,100,518.67,642.48,1591.55,1407.27,...,0.03,394,2388,100,38.92,23.3058,4,-0.019998,0.495001,0
3,4,9,4,0.0033,-0.0004,100,518.67,642.15,1587.86,1406.12,...,0.03,394,2388,100,38.88,23.3084,5,-0.089382,0.477669,0
4,5,9,5,0.0005,-0.0003,100,518.67,642.47,1591.80,1407.10,...,0.03,389,2388,100,38.85,23.3685,6,-0.185455,0.453769,0


# cluster0

In [7]:
#結果表の作成 
test_data_R_cluster0_0_5_pred = test_data_R_cluster0_0_5.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_time_flag_step'のみ
test_data_R_cluster0_0_5_pred = test_data_R_cluster0_0_5_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [8]:
test_data_R_cluster0_0_5_pred.head()

,engine_no,pred_test2_time_flag_step
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


# 上記で読み込んだtest_data_RUL_cluster0を使っていく

In [9]:
test_data_RUL_cluster0.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [10]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_5_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [11]:
cm

array([[244,  71],
       [ 59, 133]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [12]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_5_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.81      0.77      0.79       315
           0       0.65      0.69      0.67       192

    accuracy                           0.74       507
   macro avg       0.73      0.73      0.73       507
weighted avg       0.75      0.74      0.74       507



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# cluster1

In [13]:
#結果表の作成 
test_data_R_cluster1_0_5_pred = test_data_R_cluster1_0_5.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_cluster1_0_5_pred = test_data_R_cluster1_0_5_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [14]:
test_data_R_cluster1_0_5_pred.head()

,engine_no,pred_test2_time_flag_step
0,9,1
1,10,0
2,15,1
3,18,0
4,26,1


# 上記で読み込んだtest_data_RUL_cluster1を使っていく

In [15]:
test_data_RUL_cluster1.head()

,engine_no,RUL,result
0,9,93,1
1,10,120,0
2,15,55,1
3,18,99,1
4,26,93,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [16]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_5_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [17]:
cm

array([[105,  30],
       [ 18,  47]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

In [18]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_5_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.85      0.78      0.81       135
           0       0.61      0.72      0.66        65

    accuracy                           0.76       200
   macro avg       0.73      0.75      0.74       200
weighted avg       0.77      0.76      0.76       200



# 結果はそれぞれcsv出力する。一応train_data_cluster0、test_data_cluster0も出力しておく。同じ名前のファイルが既にあるので、名前を変える

In [19]:
# clusterごとに分けたデータの予測結果追加を出力
test_data_R_cluster0_0_5_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_5_pred.csv', index=False)
test_data_R_cluster1_0_5_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_5_pred.csv', index=False)

In [20]:
# clusterごとに分けたデータの予測結果追加を読み込む
test_data_R_cluster0_0_5_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_5_pred.csv')
test_data_R_cluster1_0_5_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_5_pred.csv')

# cluster0と1を結合して、test_dataの結果と比較する

In [21]:
#2つのdataを結合する
test_data_R_0_5_all = test_data_R_cluster0_0_5.append(test_data_R_cluster1_0_5)

In [22]:
test_data_R_0_5_all.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,0


In [23]:
print(test_data_R_0_5_all.shape)
print(test_data.shape)

(104897, 31)
(104897, 32)


In [24]:
# いらない列削除
test_data_R_0_5_all = test_data_R_0_5_all.drop('Unnamed: 0', axis=1)

In [25]:
# 2つのdataを最初の形に並べ替える
test_data_R_0_5_all = test_data_R_0_5_all.sort_values(['engine_no', 'time_in_cycles'])

In [26]:
# 2つのtdataを結合したデータの予測結果追加を出力
test_data_R_0_5_all.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_0.5_all.csv', index=False)

# test_dataのRULを読み込んで処理

In [27]:
#testデータのRULを読み込む
test_data_RUL = pd.read_csv('Downloads/インテグ課題/NASA/prehackathonsup/RUL_test.csv')

In [28]:
test_data_RUL.head()

,Unnamed: 0,RUL,engine_no
0,0,22,0
1,1,12,1
2,2,108,2
3,3,88,3
4,4,10,4


In [29]:
#列の順番を入れ替えと削除
test_data_RUL.drop('Unnamed: 0', axis=1)
test_data_RUL = test_data_RUL.loc[:,['engine_no','RUL']]
test_data_RUL.head()

,engine_no,RUL
0,0,22
1,1,12
2,2,108
3,3,88
4,4,10


In [30]:
# RUL<100の時、つまり予測した残りの寿命サイクルが100より小さければ1 それ以外で0とする
test_data_RUL['result'] = test_data_RUL['RUL'].map(lambda x: 0 if x > 100 else 1)

In [31]:
test_data_RUL.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


In [32]:
print((test_data_RUL['result'] == 1).sum())
print((test_data_RUL['result'] == 0).sum())

450
257


# Rで作成し、上記で結合したDataを使う。列名はresultではないので、ランダムフォレストと少し形が違う

In [33]:
#結果表の作成 
test_data_R_0_5_all_result = test_data_R_0_5_all.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_0_5_all_result = test_data_R_0_5_all.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [34]:
print((test_data_R_0_5_all_result['pred_test2_time_flag_step'] == 1).sum())
print((test_data_R_0_5_all_result['pred_test2_time_flag_step'] == 0).sum())

426
281


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [35]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_0_5_all_result['pred_test2_time_flag_step'],labels=[1, 0] )

In [36]:
cm

array([[349, 101],
       [ 77, 180]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [37]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_0_5_all_result['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.82      0.78      0.80       450
           0       0.64      0.70      0.67       257

    accuracy                           0.75       707
   macro avg       0.73      0.74      0.73       707
weighted avg       0.75      0.75      0.75       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# 閾値0.4

# Rで作成した予測付きのtestデータを読み込む

In [38]:
# clusterごとのRを読み込み
test_data_R_cluster0_0_4 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster0_0.4.csv')
test_data_R_cluster1_0_4 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster1_0.4.csv')

In [39]:
test_data_R_cluster0_0_4.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,1
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,1


In [40]:
test_data_R_cluster1_0_4.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,9,1,-0.0009,-0.0001,100,518.67,642.06,1590.15,1408.02,...,0.03,393,2388,100,38.90,23.3147,2,-0.119871,0.470068,1
1,2,9,2,0.0031,-0.0002,100,518.67,642.74,1584.08,1413.33,...,0.03,392,2388,100,38.70,23.3071,3,0.035570,0.508891,1
2,3,9,3,0.0023,-0.0001,100,518.67,642.48,1591.55,1407.27,...,0.03,394,2388,100,38.92,23.3058,4,-0.019998,0.495001,1
3,4,9,4,0.0033,-0.0004,100,518.67,642.15,1587.86,1406.12,...,0.03,394,2388,100,38.88,23.3084,5,-0.089382,0.477669,1
4,5,9,5,0.0005,-0.0003,100,518.67,642.47,1591.80,1407.10,...,0.03,389,2388,100,38.85,23.3685,6,-0.185455,0.453769,1


# cluster0

In [41]:
#結果表の作成 
test_data_R_cluster0_0_4_pred = test_data_R_cluster0_0_4.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_cluster0_0_4_pred = test_data_R_cluster0_0_4_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [42]:
test_data_R_cluster0_0_4_pred.head()

,engine_no,pred_test2_time_flag_step
0,0,1
1,1,1
2,2,0
3,3,1
4,4,1


# 上記で読み込んだtest_data_RUL_cluster0を使っていく

In [43]:
test_data_RUL_cluster0.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [44]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_4_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [45]:
cm

array([[289,  26],
       [126,  66]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [46]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_4_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.70      0.92      0.79       315
           0       0.72      0.34      0.46       192

    accuracy                           0.70       507
   macro avg       0.71      0.63      0.63       507
weighted avg       0.70      0.70      0.67       507



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# cluster1

In [47]:
#結果表の作成 
test_data_R_cluster1_0_4_pred = test_data_R_cluster1_0_4.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_cluster1_0_4_pred = test_data_R_cluster1_0_4_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [48]:
test_data_R_cluster1_0_4_pred.head()

,engine_no,pred_test2_time_flag_step
0,9,1
1,10,0
2,15,1
3,18,1
4,26,1


# 上記で読み込んだtest_data_RUL_cluster1を使っていく

In [49]:
test_data_RUL_cluster1.head()

,engine_no,RUL,result
0,9,93,1
1,10,120,0
2,15,55,1
3,18,99,1
4,26,93,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [50]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_4_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [51]:
cm

array([[130,   5],
       [ 39,  26]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

In [52]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_4_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.77      0.96      0.86       135
           0       0.84      0.40      0.54        65

    accuracy                           0.78       200
   macro avg       0.80      0.68      0.70       200
weighted avg       0.79      0.78      0.75       200



# 結果はそれぞれcsv出力している。一応train_data_cluster0、test_data_cluster0も出力しておく。同じ名前のファイルが既にあるので、名前を変える

In [53]:
# clusterごとに分けたデータの予測結果追加を出力
test_data_R_cluster0_0_4_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_4_pred.csv', index=False)
test_data_R_cluster1_0_4_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_4_pred.csv', index=False)

In [54]:
# clusterごとに分けたデータの予測結果追加を読み込む
test_data_R_cluster0_0_4_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_4_pred.csv')
test_data_R_cluster1_0_4_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_4_pred.csv')

# cluster0と1を結合して、test_dataの結果と比較する

In [55]:
#2つのdataを結合する
test_data_R_0_4_all = test_data_R_cluster0_0_4.append(test_data_R_cluster1_0_4)

In [56]:
test_data_R_0_4_all.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,1
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,1


In [91]:
print(test_data_R_0_4_all.shape)
print(test_data.shape)

(104897, 30)
(104897, 32)


In [58]:
# いらない列削除
test_data_R_0_4_all = test_data_R_0_4_all.drop('Unnamed: 0', axis=1)

In [59]:
# 2つのdataを最初の形に並べ替える
test_data_R_0_4_all = test_data_R_0_4_all.sort_values(['engine_no', 'time_in_cycles'])


In [60]:
#2つのtdataを結合したデータの予測結果追加を出力
test_data_R_0_4_all.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_0.4_all.csv', index=False)


In [61]:
print((test_data_RUL['result'] == 1).sum())
print((test_data_RUL['result'] == 0).sum())

450
257


# Rで作成し、上記で結合したDataを使う。　列名はresultではないので、ランダムフォレストと少し形が違う

In [62]:
#結果表の作成 
test_data_R_0_4_all_result = test_data_R_0_4_all.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_0_4_all_result = test_data_R_0_4_all.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [63]:
print((test_data_R_0_4_all_result['pred_test2_time_flag_step'] == 1).sum())
print((test_data_R_0_4_all_result['pred_test2_time_flag_step'] == 0).sum())

584
123


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [64]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_0_4_all_result['pred_test2_time_flag_step'],labels=[1, 0] )

In [65]:
cm

array([[419,  31],
       [165,  92]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [67]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_0_4_all_result['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.72      0.93      0.81       450
           0       0.75      0.36      0.48       257

    accuracy                           0.72       707
   macro avg       0.73      0.64      0.65       707
weighted avg       0.73      0.72      0.69       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# 閾値0.6

# Rで作成した予測付きのtestデータを読み込む

In [68]:
# clusterごとのtRを読み込み
test_data_R_cluster0_0_6 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster0_0.6.csv')
test_data_R_cluster1_0_6 = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_time_cluster1_0.6.csv')

In [69]:
test_data_R_cluster0_0_6.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,0


In [70]:
test_data_R_cluster1_0_6.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,9,1,-0.0009,-0.0001,100,518.67,642.06,1590.15,1408.02,...,0.03,393,2388,100,38.90,23.3147,2,-0.119871,0.470068,0
1,2,9,2,0.0031,-0.0002,100,518.67,642.74,1584.08,1413.33,...,0.03,392,2388,100,38.70,23.3071,3,0.035570,0.508891,0
2,3,9,3,0.0023,-0.0001,100,518.67,642.48,1591.55,1407.27,...,0.03,394,2388,100,38.92,23.3058,4,-0.019998,0.495001,0
3,4,9,4,0.0033,-0.0004,100,518.67,642.15,1587.86,1406.12,...,0.03,394,2388,100,38.88,23.3084,5,-0.089382,0.477669,0
4,5,9,5,0.0005,-0.0003,100,518.67,642.47,1591.80,1407.10,...,0.03,389,2388,100,38.85,23.3685,6,-0.185455,0.453769,0


# cluster0

In [71]:
#結果表の作成 
test_data_R_cluster0_0_6_pred = test_data_R_cluster0_0_6.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_cluster0_0_6_pred = test_data_R_cluster0_0_6_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [72]:
test_data_R_cluster0_0_6_pred.head()

,engine_no,pred_test2_time_flag_step
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


# 上記で読み込んだtest_data_RUL_cluster0を使っていく

In [73]:
test_data_RUL_cluster0.head()

,engine_no,RUL,result
0,0,22,1
1,1,12,1
2,2,108,0
3,3,88,1
4,4,10,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [74]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_6_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [75]:
cm

array([[172, 143],
       [  8, 184]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [76]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster0['result'],test_data_R_cluster0_0_6_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.96      0.55      0.69       315
           0       0.56      0.96      0.71       192

    accuracy                           0.70       507
   macro avg       0.76      0.75      0.70       507
weighted avg       0.81      0.70      0.70       507



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方

# cluster1

In [77]:
#結果表の作成 
test_data_R_cluster1_0_6_pred = test_data_R_cluster1_0_6.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_cluster1_0_6_pred = test_data_R_cluster1_0_6_pred.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [78]:
test_data_R_cluster1_0_6_pred.head()

,engine_no,pred_test2_time_flag_step
0,9,0
1,10,0
2,15,1
3,18,0
4,26,0


# 上記で読み込んだtest_data_RUL_cluster1を使っていく

In [79]:
test_data_RUL_cluster1.head()

,engine_no,RUL,result
0,9,93,1
1,10,120,0
2,15,55,1
3,18,99,1
4,26,93,1


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [82]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_6_pred['pred_test2_time_flag_step'],labels=[1, 0] )

In [83]:
cm

array([[72, 63],
       [ 2, 63]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [85]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL_cluster1['result'],test_data_R_cluster1_0_6_pred['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.97      0.53      0.69       135
           0       0.50      0.97      0.66        65

    accuracy                           0.68       200
   macro avg       0.74      0.75      0.67       200
weighted avg       0.82      0.68      0.68       200



# 結果はそれぞれcsv出力している。一応train_data_cluster0、test_data_cluster0も出力しておく。同じ名前のファイルが既にあるので、名前を変える

In [86]:
#clusterごとに分けたデータの予測結果追加をを出力
test_data_R_cluster0_0_6_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_6_pred.csv', index=False)
test_data_R_cluster1_0_6_pred.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_6_pred.csv', index=False)

In [87]:
#clusterごとに分けたデータの予測結果追加を読み込む
test_data_R_cluster0_0_6_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster0_0_6_pred.csv')
test_data_R_cluster1_0_6_pred = pd.read_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_cluster1_0_6_pred.csv')

# cluster0と1を結合して、test_dataの結果と比較する

In [88]:
#2つのdataを結合する
test_data_R_0_6_all = test_data_R_cluster0_0_6.append(test_data_R_cluster1_0_6)

In [89]:
test_data_R_0_6_all.head()

,Unnamed: 0,engine_no,time_in_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,...,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,stop,pred_test_time_step,pred_test2_time_step,pred_test2_time_flag_step
0,1,0,1,42.0034,0.8400,100,445.00,549.36,1342.05,1124.56,...,0.02,329,2212,100.0,10.69,6.3956,2,-0.553710,0.365004,0
1,2,0,2,42.0017,0.8400,100,445.00,548.83,1351.93,1116.28,...,0.02,331,2212,100.0,10.55,6.3775,3,-0.555234,0.364651,0
2,3,0,3,0.0028,0.0019,100,518.67,642.35,1583.74,1400.44,...,0.03,392,2388,100.0,38.85,23.3483,4,-0.410151,0.398876,0
3,4,0,4,42.0047,0.8400,100,445.00,549.69,1354.36,1125.55,...,0.02,330,2212,100.0,10.56,6.4871,5,-0.360648,0.410803,0
4,5,0,5,10.0058,0.2506,100,489.05,604.72,1496.65,1310.52,...,0.03,370,2319,100.0,28.78,17.1987,6,-0.358974,0.411208,0


In [92]:
print(test_data_R_0_6_all.shape)
print(test_data.shape)

(104897, 31)
(104897, 32)


In [93]:
#いらない列削除
test_data_R_0_6_all = test_data_R_0_6_all.drop('Unnamed: 0', axis=1)

In [94]:
# 2つのdataを最初の形に並べ替える
test_data_R_0_6_all = test_data_R_0_6_all.sort_values(['engine_no', 'time_in_cycles'])


In [95]:
#2つのtdataを結合したデータの予測結果追加を出力
test_data_R_0_6_all.to_csv('Desktop/インテグ課題/NASA/test_data/test_data_R_0.6_all.csv', index=False)


In [96]:
print((test_data_RUL['result'] == 1).sum())
print((test_data_RUL['result'] == 0).sum())

450
257


# Rで作成し、上記で結合したDataを使う。　列名はresultではないので、ランダムフォレストと少し形が違う

In [97]:
#結果表の作成 
test_data_R_0_6_all_result = test_data_R_0_6_all.sort_values(['engine_no', 'time_in_cycles'])

#test_data_Rの各engineの最後のサイクルの行のみを表示　列は'engine_no', 'pred_test2_flag'のみ
test_data_R_0_6_all_result = test_data_R_0_6_all.groupby('engine_no').last().reset_index()[['engine_no', 'pred_test2_time_flag_step']]



In [98]:
print((test_data_R_0_6_all_result['pred_test2_time_flag_step'] == 1).sum())
print((test_data_R_0_6_all_result['pred_test2_time_flag_step'] == 0).sum())

254
453


# 100以内に故障するを1,しないを0としている 故障することを当てたいから、故障する1をpositiveと定義する。

In [99]:
#混合行列
#1 = Positive,0 = Negativeとする　なぜならば、再現率などがpositiveで計算されるので、故障するをpositiveにする
#labels指定をしないと、左から0,1表記になるので、TPが右下になってしまう。
cm = confusion_matrix(test_data_RUL['result'],test_data_R_0_6_all_result['pred_test2_time_flag_step'],labels=[1, 0] )

In [100]:
cm

array([[244, 206],
       [ 10, 247]], dtype=int64)

# 列が予測の1と0行が実際の1と0つまり、真陽性 : TP (True-Positive)332、真陰性: TN (True-Negative)239、偽陽性: FP (False-Positive)18(故障していないのにしていると予測)、偽陰性: FN (False-Negative)118(実際は故障しているのに故障していないと予測)
#https://pythondatascience.plavox.info/scikit-learn/%E5%88%86%E9%A1%9E%E7%B5%90%E6%9E%9C%E3%81%AE%E3%83%A2%E3%83%87%E3%83%AB%E8%A9%95%E4%BE%A1
#FN (False-Negative)118を少なくしたい　つまり実際は故障だが、故障しないと判断してしまっているものを

In [101]:
#混合行列まとめて表示
cr = classification_report(test_data_RUL['result'],test_data_R_0_6_all_result['pred_test2_time_flag_step'],labels=[1, 0] )
print(cr)

              precision    recall  f1-score   support

           1       0.96      0.54      0.69       450
           0       0.55      0.96      0.70       257

    accuracy                           0.69       707
   macro avg       0.75      0.75      0.69       707
weighted avg       0.81      0.69      0.69       707



# labelsを付けた時の計算が上記のまとめて表示の1の方　labelsをつけなかったときの計算が下の方